In [2]:
import duckdb

In [3]:
db_path = '/workspaces/dw-lantern/lantern_dw/transactions.duckdb'

In [26]:

conn = duckdb.connect(db_path)

df = conn.execute("select * from fund_info order by fund").fetchdf()

display(df)

conn.close()

,company_id,company,fund,invested
0,2,ModuGear,Fund 1,100000
1,4,OmniCorp,Fund 1,100000
2,1,FreshByte,Fund 2,10000
3,5,TitanTech,Fund 2,100000
4,3,NanoBuild,Fund 2,50000


In [5]:

conn = duckdb.connect(db_path)

df = conn.execute("select * from transactions where company = 'ModuGear'").fetchdf()

display(df)

conn.close()

,company,date,transaction_type,amount,number
0,ModuGear,2024-01-01,Buy Item Cost,34800.0,58
1,ModuGear,2024-01-01,Sales of Item,810.0,1
2,ModuGear,2024-01-02,Sales of Item,3240.0,4
3,ModuGear,2024-01-03,Sales of Item,810.0,1
4,ModuGear,2024-01-05,Sales of Item,3240.0,4
...,...,...,...,...,...
191,ModuGear,2024-12-08,Sales of Item,810.0,1
192,ModuGear,2024-12-10,Upkeep Cost,5000.0,1
193,ModuGear,2024-12-19,Sales of Item,2430.0,3
194,ModuGear,2024-12-27,Sales of Item,4050.0,5


In [20]:

conn = duckdb.connect(db_path)

df = conn.execute("select company, transaction_type, max(amount), min(amount), max(number), min(number) from transactions group by company, transaction_type order by transaction_type").fetchdf()

display(df)

conn.close()

,company,transaction_type,max(amount),min(amount),max(number),min(number)
0,FreshByte,Buy Item Cost,48000.0,10080.0,100,21
1,ModuGear,Buy Item Cost,41400.0,9600.0,69,16
2,NanoBuild,Buy Item Cost,46530.0,7520.0,99,16
3,OmniCorp,Buy Item Cost,55500.0,39500.0,111,79
4,TitanTech,Buy Item Cost,62000.0,41000.0,124,82
5,FreshByte,Sales of Item,3840.0,768.0,5,1
6,ModuGear,Sales of Item,4050.0,810.0,5,1
7,NanoBuild,Sales of Item,3995.0,799.0,5,1
8,OmniCorp,Sales of Item,3500.0,700.0,5,1
9,TitanTech,Sales of Item,3750.0,750.0,5,1


In [22]:

conn = duckdb.connect(db_path)

df = conn.execute("select transaction_type, count(1), max(amount), min(amount), max(number), min(number) from transactions group by transaction_type order by transaction_type").fetchdf()

display(df)

conn.close()

,transaction_type,count(1),max(amount),min(amount),max(number),min(number)
0,Buy Item Cost,60,62000.0,7520.0,124,16
1,Sales of Item,1388,4050.0,700.0,5,1
2,Upkeep Cost,60,5000.0,800.0,1,1


In [11]:

conn = duckdb.connect(db_path)

df = conn.execute("select company, transaction_type, sum(amount) from transactions group by company, transaction_type, number order by transaction_type, number having transaction_type").fetchdf()

display(df)

conn.close()

,company,transaction_type,sum(amount)
0,NanoBuild,Buy Item Cost,7520.0
1,ModuGear,Buy Item Cost,9600.0
2,ModuGear,Buy Item Cost,10800.0
3,ModuGear,Buy Item Cost,12600.0
4,FreshByte,Buy Item Cost,10080.0
...,...,...,...
76,FreshByte,Upkeep Cost,12000.0
77,NanoBuild,Upkeep Cost,9600.0
78,TitanTech,Upkeep Cost,60000.0
79,ModuGear,Upkeep Cost,60000.0


In [8]:

conn = duckdb.connect(db_path)

df = conn.execute("""
    WITH company_cash AS (
    SELECT 
        company_cleaned AS company,
        SUM(CASE WHEN transaction_type = 'Sales of Item' THEN number ELSE 0 END) AS inflow,
        SUM(CASE WHEN transaction_type IN ('Buy Item Cost', 'Upkeep Cost') THEN number ELSE 0 END) AS outflow
    FROM "01_cleaned_transactions"
    GROUP BY company_cleaned
),
overall_fund AS (
    SELECT 
        fund, 
        SUM(invested) AS total_invested
    FROM "01_cleaned_fund_info"
    GROUP BY fund
),
fund_performance_1 AS (
    SELECT 
        f.company_cleaned AS company, 
        f.fund, 
        f.invested,
        c.inflow,
        c.outflow
    FROM "01_cleaned_fund_info" f
    JOIN company_cash c 
        ON f.company_cleaned = c.company
)
SELECT *,
    invested + inflow - outflow as net_investment           
FROM fund_performance_1
"""
).fetchdf()
        

display(df)

conn.close()

,company,fund,invested,inflow,outflow,net_investment
0,FreshByte,Fund 2,10000,819.0,832.0,9987.0
1,ModuGear,Fund 1,100000,540.0,553.0,99987.0
2,OmniCorp,Fund 1,100000,1130.0,1164.0,99966.0
3,The Titan Tech,Fund 2,100000,1208.0,1250.0,99958.0
4,NanoBuild,Fund 2,50000,823.0,846.0,49977.0


In [15]:
conn = duckdb.connect(db_path)

df = conn.execute("select *, total_revenue - total_cost as profit from '02_updated_company_report' where company in ('ModuGear', 'OmniCorp') order by company, quarter_start_date").fetchdf()

display(df)
conn.close()

,company_id,company,quarter_start_date,sales_amount,buy_cost_amount,total_cost,total_revenue,profit
0,2,ModuGear,2024-01-01,148230.0,109800.0,124800.0,148230.0,23430.0
1,2,ModuGear,2024-04-01,128790.0,95400.0,110400.0,128790.0,18390.0
2,2,ModuGear,2024-07-01,102870.0,76200.0,91200.0,102870.0,11670.0
3,2,ModuGear,2024-10-01,44550.0,33000.0,48000.0,44550.0,-3450.0
4,2,ModuGear,2025-01-01,12960.0,9600.0,11600.0,12960.0,1360.0
5,4,OmniCorp,2024-01-01,173600.0,137000.0,151400.0,173600.0,22200.0
6,4,OmniCorp,2024-04-01,190400.0,135500.0,149900.0,190400.0,40500.0
7,4,OmniCorp,2024-07-01,175000.0,123500.0,137900.0,175000.0,37100.0
8,4,OmniCorp,2024-10-01,188300.0,126500.0,140900.0,188300.0,47400.0
9,4,OmniCorp,2025-01-01,63700.0,53000.0,57800.0,63700.0,5900.0


In [16]:
conn = duckdb.connect(db_path)

df = conn.execute("select *, total_revenue - total_cost as profit from '02_updated_company_report' where company not in ('ModuGear', 'OmniCorp') order by company, quarter_start_date").fetchdf()

display(df)
conn.close()

,company_id,company,quarter_start_date,sales_amount,buy_cost_amount,total_cost,total_revenue,profit
0,1,FreshByte,2024-01-01,68352.0,42720.0,45720.0,68352.0,22632.0
1,1,FreshByte,2024-04-01,110592.0,69120.0,72120.0,110592.0,38472.0
2,1,FreshByte,2024-07-01,181248.0,113760.0,116760.0,181248.0,64488.0
3,1,FreshByte,2024-10-01,197376.0,122880.0,125880.0,197376.0,71496.0
4,1,FreshByte,2025-01-01,71424.0,44640.0,45640.0,71424.0,25784.0
5,3,NanoBuild,2024-01-01,62322.0,36660.0,39060.0,62322.0,23262.0
6,3,NanoBuild,2024-04-01,125443.0,73790.0,76190.0,125443.0,49253.0
7,3,NanoBuild,2024-07-01,180574.0,112800.0,115200.0,180574.0,65374.0
8,3,NanoBuild,2024-10-01,210936.0,119380.0,121780.0,210936.0,89156.0
9,3,NanoBuild,2025-01-01,78302.0,48880.0,49680.0,78302.0,28622.0
